In [2]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer 
# this is used as each row has different labels of the same category 


In [3]:
# loading the cleaned datasets and not the original one as we are using modular approach
df=pd.read_csv('cleaned_data.csv')


In [ ]:
# feature engineering i.e. encoding categorical to numerical
df['Genre_list']=df['Genre'].apply(lambda x: list(dict.fromkeys(i.strip() for i in x.split(','))))
'''
dict.fromkeys is used to remove the duplicates as in dictionary no 2 keys can be same
and list () is used to convert this to list
strip to remove any spaces 
this line just give me the list of the grenre across all the rows i.e. removes repetioon on the rows but not on whole datasets
but multilabelbinarizer only makes columns of the unique list
'''

# multilabelizer for genre
mlb_genre =MultiLabelBinarizer()
Genre_encoded =pd.DataFrame(mlb_genre.fit_transform(df['Genre_list']), columns=mlb_genre.classes_)
'''
fit_transform() gives numpy arrays without any names or titles of columns so column name is given on the basis
of the Genre_list stored at the mlb_genre.classes_
The unique genres found in df['Genre_list']
during fit_transform() are automatically saved into mlb_genre.classes_.
'''
df = pd.concat([df, Genre_encoded], axis=1)


print(Genre_encoded)
print(mlb_genre.classes_)

In [ ]:
# now for the director 
top=df['Director'].value_counts().nlargest(100).index